In [ ]:
if True:
    import polars as pl
    import plotly.express as px
    import plotly.graph_objects as go
    pl.Config().set_tbl_rows(20)

    df = (
        pl.read_csv('Halloween.csv')
        .select(
            YEAR = pl.col('Date')
                .str.split('/')
                .list.get(2)
                # .cast(pl.Categorical),
                .cast(pl.UInt16()),
            DAY = pl.col('Day of Week').str.slice(0,3),
            TIME = pl.col('Time').str.replace('pm',' PM'),
            COUNT = pl.col('Count').cast(pl.UInt8()),
        )
    )
    time_list = sorted(df.unique('TIME')['TIME'].to_list())

    time_color_dict = {}
    for i, time in enumerate(time_list):
        # time_color_dict[time] = px.colors.qualitative.Light24[i]
        time_color_dict[time] = px.colors.qualitative.Alphabet[i]
        time_color_dict[time] = px.colors.qualitative.D3[i]
        
    time_color_dict

df

AttributeError: 'DataFrame' object has no attribute 'uniq4ue'

## Here is the first graph


In [ ]:
if True:
    fig=px.line(
        df,
        x='YEAR',
        y='COUNT',
        color='TIME',
        color_discrete_map=time_color_dict,
        template = 'simple_white',
        markers=True,
        title='Halloween TTT',
        height=400, width=800
    )
    fig.update_layout(
        xaxis=dict(
            showline=True, 
            linewidth=1, 
            linecolor='gray', 
            mirror=True,
            showgrid=True,      # show vertical grid lines
            tickmode='linear',  # ensure ticks are evenly spaced
            dtick=1             # one tick/gridline per category
            ),
        yaxis=dict(
            showline=True, 
            linewidth=1, 
            linecolor='gray', 
            mirror=True,
            showgrid=True       # show horizontal grid lines
            ),
        title_y=0.97,
    )
    fig.update_traces(line=dict( width=1))

    # Customize gridlines
    fig.update_xaxes(
        showgrid=True,           # Ensure gridlines are visible
        gridwidth=1,             # Thickness of vertical gridlines
        gridcolor='gray',        # Color of vertical gridlines
    )

    fig.update_yaxes(
        showgrid=True,          # Ensure gridlines are visible
        gridwidth=1,            # Thickness of horizontal gridlines
        gridcolor='gray',        # Color of horizontal gridlines
    )
    fig.update_layout(margin=dict(l=0, r=0, t=50, b=0))

    # Show the figure
    fig.show()

## Fix the title

In [ ]:
if True:
    fig.update_layout(title=dict(text='Halloween Trick-or-Treaters by Time (TTT)'))
    fig.show()

## change change line shape, linear to spline

In [ ]:
if True:
    fig.update_traces(line=dict(shape='spline'))
    fig.show()

## Let's lose the grid lines


In [ ]:
if True:
    fig.update_xaxes(showgrid=False, zeroline=False)
    fig.update_yaxes(showgrid=False, zeroline=False)
    fig.show()

## remove top and right side borders

In [ ]:
if True:
    fig.update_layout(
        xaxis=dict(showline=True, showgrid=False, zeroline=False, mirror=False),
        yaxis=dict(showline=True, showgrid=False, zeroline=False, mirror=False),
        plot_bgcolor='white',
    )
    fig.update_layout(margin=dict(l=0, r=0, t=30, b=0))
    fig.show()

## Drop the word Year from the x-axis label
Because it is obvious, it qualifies as chart junk

In [ ]:
if True:
    fig.update_xaxes(title='')
    fig.show()

## reduce the density of x-axis year labels

In [ ]:
if True:
    fig.update_layout(
        xaxis=dict(
            showline=True, 
            mirror=False,
            showgrid=False,      # show vertical grid lines
            tickmode='linear',  # ensure ticks are evenly spaced
            dtick=2             # one tick/gridline per category
            ),
        yaxis=dict(
            showline=True, 
            mirror=False,
            showgrid=False       # show horizontal grid lines
            ),
    )
    fig.show()

## remove legend

In [ ]:
if True:
    fig.update_layout(showlegend=False)
    fig.show()

## use trace annotations in place of the legend

In [ ]:
if True:
    count_list = df.filter(pl.col('YEAR') == 2024)['COUNT'].to_list()
    print(count_list)
    fig.update_xaxes(range=[2008, 2025.75])
    y_shift_list = [15, 10, 10, 10, -15, -15]
    for i, shift in enumerate(y_shift_list):
        fig.add_annotation(
            x=1,xref='paper', 
            y=count_list[i], yref='y', 
            text=time_list[i], showarrow=False, 
            font=dict(color=time_color_dict[time_list[i]], size=14), 
            yshift=y_shift_list[i])
    fig.show()

[34, 109, 203, 141, 111, 33]


## Add a vertical line to denoting Covid 19

In [ ]:
if True:
    fig.add_vline(
        x=2020, 
        line_width=2, 
        line_dash='dash',
        line_color='gray',
         annotation_text='Covid-19 Pandemic',
    )
    fig.show()

## plot counts per year

In [ ]:
if True:
    df_yearly = (
        df
        .group_by(['YEAR'])
        .agg(pl.sum('COUNT').alias('TOTAL_COUNT'))
        .sort('YEAR')
    )
    count_2022 = df_yearly.filter(pl.col('YEAR') == 2022).item(0, 'TOTAL_COUNT')
    count_2024 = df_yearly.filter(pl.col('YEAR') == 2024).item(0, 'TOTAL_COUNT')
    count_2025 = int(count_2024 + 0.5*(count_2024 - count_2022))
    # print(f'{count_2022 = }')
    # print(f'{count_2024 = }')
    # print(f'{count_2025 = }')
    df_future = pl.DataFrame({
        'YEAR':           [2024, 2025],
        'TOTAL_COUNT' :   [count_2024, count_2025]
    })
    # print(df_future)
    # print(df_yearly)
    fig=px.line(
        df_yearly,
        x='YEAR',
        y='TOTAL_COUNT',
        template = 'simple_white',
        markers=True,
        title='Total Halloween Trick-or-Treaters by Year',
        height=400, width=800,
        line_shape='spline',
    )
    fig.update_traces(line=dict( width=1))
    fig.update_layout(margin=dict(l=0, r=0, t=50, b=0))
    fig.add_vline(
        x=2020, 
        line_width=2, 
        line_dash='dash',
        line_color='gray',
        annotation_text='Covid-19 Pandemic',
    )
    fig.add_vline(
        x=2013, 
        line_width=2, 
        line_dash='dash',
        line_color='gray',
        annotation_text='1 inch of rain',
    )
    for i, year in enumerate([2022, 2023, 2024]):
        year_count = df_yearly.filter(pl.col('YEAR') == year).item(0, 'TOTAL_COUNT')
        fig.add_annotation(
            x=year,xref='x', 
            y=year_count, yref='y', 
            text=f'{year_count}', showarrow=False, 
            font=dict(color='gray', size=14), 
            yshift=20
            )
    fig.update_xaxes(title='')
    fig.add_trace(go.Scatter(
        x=df_future['YEAR'],
        y=df_future['TOTAL_COUNT'],
        mode='lines+markers',
        line=dict(
            color='gray',
            dash='dot'  # Options: 'solid', 'dot', 'dash', 'longdash', 'dashdot', 'longdashdot'
        ),
        marker=dict(size=8),
        name='Gray Dashed Line',
        # zorder=10,
        showlegend=False
    ))
    fig.add_annotation(
        x=2025,xref='x', 
        y=count_2025, yref='y', 
        text=f'{count_2025}', showarrow=False, 
        font=dict(color='gray', size=14), 
        yshift=20
        )
    fig.update_layout(
        # margin=dict(l=0, r=0, t=50, b=0),
        xaxis=dict(
            showline=True, 
            linewidth=1, 
            linecolor='gray',
            mirror=False, 
            ),
        yaxis=dict(
            showline=True, 
            linewidth=1, 
            linecolor='gray', 
            mirror=False     
            ),
        title_y=0.97,
    )
    # Show the figure
    fig.show()

## plot counts per day of week

In [ ]:
if True:
    df_ordered_days = (
        pl.DataFrame({
            'DAY'     : ['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat'],
            'DAY_NUM' : [    1,     2,     3,     4,     5,     6,     7],
            'DAY_COLOR' : ['#d3d3d3']*5 + ['orange', '#d3d3d3'],
        })
    )
    dict_day_color = dict(zip(
        df_ordered_days['DAY'], 
        df_ordered_days['DAY_COLOR']
    ))
    print(dict_day_color)

    df_avg_by_day = (
        df
        .with_columns(DAY_COUNT = (pl.col('DAY').len().over('DAY')/6))
        .with_columns(NORM_DAY_COUNT = (pl.col('COUNT')/(pl.col('DAY_COUNT'))))
        .with_columns(DAY_TOTAL = (pl.col('NORM_DAY_COUNT').sum().over('DAY')))
        .unique(['DAY', 'DAY_TOTAL'])
        .select(['DAY', 'DAY_TOTAL'])
        .join(
            df_ordered_days,
            on='DAY',
            how='left'
        )
        .sort('DAY_NUM')
    )
    fig=px.histogram(
        df_avg_by_day,
        x='DAY',
        y='DAY_TOTAL',
        color='DAY',
        color_discrete_map=dict_day_color,
        template = 'simple_white',
        title='<b>Call to Action:</b> Halloween 2025 is on a Friday -- time to shop',
        height=400, width=800,
    )
    fig.update_layout(
        xaxis=dict(
            showline=True, 
            linewidth=1, 
            linecolor='gray', 
            mirror=False,
            showgrid=False,      # show vertical grid lines
            tickmode='linear',  # ensure ticks are evenly spaced
            dtick=1             # one tick/gridline per category
            ),
        yaxis=dict(
            showline=True, 
            linewidth=1, 
            linecolor='gray', 
            mirror=False,
            showgrid=False       # show horizontal grid lines
            ),
        title_y=0.97,
    )
    fig.update_xaxes(title='')
    fig.update_yaxes(title='Avg # Trick or Treaters by Day')

    fig.update_layout(margin=dict(l=0, r=0, t=50, b=0),showlegend=False)
    fig.show()

{'Sun': '#d3d3d3', 'Mon': '#d3d3d3', 'Tue': '#d3d3d3', 'Wed': '#d3d3d3', 'Thu': '#d3d3d3', 'Fri': 'orange', 'Sat': '#d3d3d3'}


In [ ]:
print(df['DAY'].value_counts())

shape: (7, 2)
┌─────┬───────┐
│ DAY ┆ count │
│ --- ┆ ---   │
│ str ┆ u32   │
╞═════╪═══════╡
│ Fri ┆ 12    │
│ Sat ┆ 18    │
│ Mon ┆ 18    │
│ Wed ┆ 12    │
│ Sun ┆ 12    │
│ Thu ┆ 18    │
│ Tue ┆ 12    │
└─────┴───────┘


In [ ]:
df_ordered_days = (
    pl.DataFrame({
        'DAY'     : ['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat'],
        'DAY_NUM' : [    1,     2,     3,     4,     5,     6,     7]
    })
)

df_avg_by_day = (
    df
    .with_columns(DAY_COUNT = (pl.col('DAY').len().over('DAY')/6))
    .with_columns(NORM_DAY_COUNT = (pl.col('COUNT')/(pl.col('DAY_COUNT'))))
    .with_columns(DAY_TOTAL = (pl.col('NORM_DAY_COUNT').sum().over('DAY')))
    .unique(['DAY', 'DAY_TOTAL'])
    .select(['DAY', 'DAY_TOTAL'])
    .join(
        df_ordered_days,
        on='DAY',
        how='left'
    )
    .sort('DAY_NUM')
)
df_avg_by_day

DAY,DAY_TOTAL,DAY_NUM
str,f64,i64
"""Sun""",606.5,1
"""Mon""",734.333333,2
"""Tue""",673.0,3
"""Wed""",636.5,4
"""Thu""",515.0,5
"""Fri""",473.0,6
"""Sat""",502.666667,7


In [ ]:
df_ordered_days

DAY,DAY_NUM
str,i64
"""Sun""",1
"""Mon""",2
"""Tue""",3
"""Wed""",4
"""Thu""",5
"""Fri""",6
"""Sat""",7


In [ ]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('Halloween.csv')
df = df.assign(Year = df['Date'].str.slice(-4))

print(df)
fig=px.line(
    df,
    x='Year', y='Count',
    color='Time',
    title='Haloween Visitor trick or treater counts by time and year'
)
fig.show()


           Date     Date and Time  Count Day of Week    Time  Year
0    10/31/2008  10/31/2008 18:00      0      Friday  6:00pm  2008
1    10/31/2008  10/31/2008 18:30     75      Friday  6:30pm  2008
2    10/31/2008  10/31/2008 19:00    117      Friday  7:00pm  2008
3    10/31/2008  10/31/2008 19:30    147      Friday  7:30pm  2008
4    10/31/2008  10/31/2008 20:00    144      Friday  8:00pm  2008
..          ...               ...    ...         ...     ...   ...
97   10/31/2024  10/31/2024 18:30    109    Thursday  6:30pm  2024
98   10/31/2024  10/31/2024 19:00    203    Thursday  7:00pm  2024
99   10/31/2024  10/31/2024 19:30    141    Thursday  7:30pm  2024
100  10/31/2024  10/31/2024 20:00    111    Thursday  8:00pm  2024
101  10/31/2024  10/31/2024 20:15     33    Thursday  8:15pm  2024

[102 rows x 6 columns]
